<a href="https://colab.research.google.com/github/smfwlgus/DnM/blob/master/%EB%86%8D%EC%82%B0%EB%AC%BC_%EC%A0%84%EC%B2%98%EB%A6%AC_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# <font color = 'skyblue'> **AT_TSALET_ALL 에서 train.csv 코드** <font>


*  private: AT_TSALET_ALL -> private_data.csv
*  test_AT_TSALET_ALL(21년 8,9,10) -> 전체 형태를 바꾸는 방식 test.csv 제작



In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import multiprocessing
import pickle

In [ ]:
# 윈도우즈 사용시 함수를 별도의 .py파일로 저장 후 import하여 사용
def preprocessing(tsalet_file):
    unique_pum = [
        '배추', '무', '양파', '건고추','마늘',
        '대파', '얼갈이배추', '양배추', '깻잎',
        '시금치', '미나리', '당근',
        '파프리카', '새송이', '팽이버섯', '토마토',
    ]
    
    unique_kind = [
        '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
    ]
    
    t_dict = {
        'date':[]
    }
    
    for sub in unique_pum:
        t_dict[f'{sub}_거래량(kg)'] = []
        t_dict[f'{sub}_가격(원/kg)'] = []
        
    for sub in unique_kind:
        t_dict[f'{sub}_거래량(kg)'] = []
        t_dict[f'{sub}_가격(원/kg)'] = []
        
    tsalet_sample = pd.read_csv(tsalet_file)
    days = sorted(tsalet_sample['SALEDATE'].unique())
    for day in days:
        t_dict['date'].append(day)
        for sub in unique_pum:
            # 날짜별, 품목별, 거래량이 0 이상인 행만 선택
            c = tsalet_sample[(tsalet_sample['SALEDATE']==day) & (tsalet_sample['PUM_NM']==sub) & (tsalet_sample['TOT_QTY']>0)]
            if c.shape[0] == 0:
                t_dict[f'{sub}_거래량(kg)'].append(0)
                t_dict[f'{sub}_가격(원/kg)'].append(0)
            else:
                tot_amt = c['TOT_AMT'].sum().astype(float)
                tot_qty = c['TOT_QTY'].sum().astype(float)
                mean_price = tot_amt/(tot_qty+1e-20)
                t_dict[f'{sub}_거래량(kg)'].append(tot_qty)
                t_dict[f'{sub}_가격(원/kg)'].append(mean_price)
        
        for sub in unique_kind:
            # 날짜별, 품종별, 거래량이 0 이상인 행만 선택
            c = tsalet_sample[(tsalet_sample['SALEDATE']==day) & (tsalet_sample['KIND_NM']==sub) & (tsalet_sample['TOT_QTY']>0)]
            if c.shape[0] == 0:
                t_dict[f'{sub}_거래량(kg)'].append(0)
                t_dict[f'{sub}_가격(원/kg)'].append(0)
            else:
                tot_amt = c['TOT_AMT'].sum().astype(float)
                tot_qty = c['TOT_QTY'].sum().astype(float)
                mean_price = round(tot_amt/(tot_qty+1e-20))
                tot_qty = round(tot_qty, 1)
                t_dict[f'{sub}_거래량(kg)'].append(tot_qty)
                t_dict[f'{sub}_가격(원/kg)'].append(mean_price)
                
    with open(f'/content/drive/MyDrive/Capstone/Data/test_AT_TSALET_ALL/{tsalet_file.split("/")[-1].split(".")[0]}.pkl', 'wb') as f:
        pickle.dump(t_dict, f)

In [ ]:
# 윈도우즈 사용시 함수를 별도의 .py파일로 저장 후 import하여 사용
def preprocessing(tsalet_file):
    unique_pum = [
        '배추', '무', '양파', '마늘', '대파', 
        '깻잎', '시금치', '미나리', '파프리카', '토마토',
    ]
    
    t_dict = {
        'date':[]
    }
    
    for sub in unique_pum:
        t_dict[f'{sub}_거래량(kg)'] = []
        t_dict[f'{sub}_가격(원/kg)'] = []
        
    tsalet_sample = pd.read_csv(tsalet_file)
    days = sorted(tsalet_sample['SALEDATE'].unique())
    for day in days:
        t_dict['date'].append(day)
        for sub in unique_pum:
            # 날짜별, 품목별, 거래량이 0 이상인 행만 선택
            c = tsalet_sample[(tsalet_sample['SALEDATE']==day) & (tsalet_sample['PUM_NM']==sub) & (tsalet_sample['TOT_QTY']>=0)]
            if c.shape[0] == 0:
                t_dict[f'{sub}_거래량(kg)'].append(0)
                t_dict[f'{sub}_가격(원/kg)'].append(0)
            else:
                tot_amt = c['TOT_AMT'].sum().astype(float)
                tot_qty = c['TOT_QTY'].sum().astype(float)
                mean_price = tot_amt/(tot_qty+1e-20)
                t_dict[f'{sub}_거래량(kg)'].append(tot_qty)
                t_dict[f'{sub}_가격(원/kg)'].append(mean_price)
                
    with open(f'/content/drive/MyDrive/Capstone/Data/test_AT_TSALET_ALL/{tsalet_file.split("/")[-1].split(".")[0]}.pkl', 'wb') as f:
        pickle.dump(t_dict, f)

In [ ]:
tsalet_files = sorted(glob('/content/drive/MyDrive/Capstone/Data/test_AT_TSALET_ALL/*'))

pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
pool.map(preprocessing, tsalet_files)
pool.close()
pool.join()

In [ ]:
dict_files = sorted(glob('/content/drive/MyDrive/Capstone/Data/test_AT_TSALET_ALL/*.pkl'))

In [ ]:
t_dict_list = []
for dict_file in dict_files:
    with open(dict_file, 'rb') as f:
        train_dict = pickle.load(f)
    t_dict_list.append(train_dict)

In [ ]:
train = None

for t_dcit in t_dict_list:
    if train is None:
        train = pd.DataFrame(t_dcit)
    else:
        train = pd.concat([train, pd.DataFrame(t_dcit)])

In [ ]:
train['date'] = train.date.astype(str).str.replace('-','')
train['date'] = pd.to_datetime(train.date, format='%Y%m%d')

NameError: ignored

In [ ]:
train #test_AT_TSALET_ALL 사용해서 test.csv -> test_files로 바꾸면 끝남

,date,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg)
0,2021-01-02,24401.50,388.534188,21461.0,559.526583,20606.0,1058.393186,2408.0,5129.931063,27859.0,2384.020245,4079.5,13150.276995,9920.00,2642.957863,7033.0,1614.409498,7849.0,4884.721875,25290.0,1914.895216
1,2021-01-04,859651.80,441.499637,1665261.7,542.826183,1823037.4,1266.627437,121200.4,5425.898627,874594.6,2202.568178,112324.7,9501.869972,374131.63,2966.183666,79020.5,3509.511798,150290.6,4515.337752,393979.0,2098.278649
2,2021-01-05,889332.20,412.451877,1647835.5,570.874791,1513826.0,1329.721668,125712.0,5147.233605,765322.2,2240.754967,69850.2,11128.609839,295523.20,3111.929321,74112.0,3965.334750,89124.0,5515.671581,228576.5,2092.442574
3,2021-01-06,1006337.30,401.758318,1787373.1,591.115374,1590580.8,1413.075900,105045.0,5392.696330,717011.6,2512.246031,68027.0,13562.134858,314793.20,3512.928332,75320.7,3905.462456,168572.5,5598.452227,163296.0,2270.461554
4,2021-01-07,535487.60,457.994775,1556727.2,617.443606,1329268.4,1474.508745,103473.0,5195.982817,688425.8,3006.010365,68557.3,11582.734209,297299.90,3689.072408,51931.5,4392.448861,74728.8,5558.861416,177869.5,2060.298264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,2021-11-26,3322119.50,855.526302,2848866.7,435.322178,1420063.1,824.988797,77781.7,6688.693844,606203.2,1159.468442,67183.0,5270.268357,229238.05,2226.226340,82529.9,6738.824341,165520.6,2108.242279,231763.5,2675.807783
25,2021-11-27,3099051.25,844.090363,2657614.8,378.614620,1354649.4,772.076177,77304.3,6706.403292,713593.0,1086.698594,67172.0,5169.389046,221894.62,2275.984041,85130.5,8013.229947,122186.3,2080.165043,196120.0,2294.284622
26,2021-11-28,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
27,2021-11-29,2856016.45,801.194158,2209466.0,380.848191,1910143.7,816.013853,84870.7,6332.339559,682166.3,1150.958392,79927.8,5897.250343,275299.90,2294.806424,82265.7,7140.467011,203478.4,2274.810506,232403.5,2486.894345


In [ ]:
train.to_csv('/content/drive/MyDrive/Capstone/Data/test.csv', index=False)

In [ ]:
train2 = pd.read_csv('/content/drive/MyDrive/Capstone/Data/test.csv')

In [ ]:
tr = train2.copy()

In [ ]:
for i in range(tr.shape[0]):
  if tr['깻잎_가격(원/kg)'][i] == 0:
    tr = tr.drop([i], axis=0)

tr

,date,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg)
1,2021-08-02,998824.04,763.819668,1140263.3,610.960763,1513622.4,665.459000,114825.2,6251.908823,506513.80,885.820829,115570.4,5330.649812,19810.86,8397.272405,49062.1,3063.390540,229907.7,1767.123850,578291.0,1340.590481
2,2021-08-03,820255.49,905.244796,1118840.6,567.260575,1589983.6,677.135062,136224.1,6121.899451,463368.70,988.611523,84821.9,4918.613153,18438.90,7488.402182,47859.5,2795.417002,143655.8,2109.013468,408103.6,1245.547699
3,2021-08-04,1000020.00,779.683542,1178932.3,492.572093,1589479.2,771.450325,97192.0,6716.579883,445159.00,1259.425235,88627.9,5448.249468,17413.44,7900.040486,44720.0,3035.610555,163340.3,2250.771383,448618.3,1231.644451
4,2021-08-05,930687.10,821.295017,1159449.5,500.682069,1981966.3,776.632317,85013.5,6125.134337,523013.30,1307.881673,92323.2,6897.028797,15223.80,8821.209488,49449.7,3597.859744,181197.5,2543.051736,410853.3,1195.879868
5,2021-08-06,1020373.75,841.846138,1121770.3,501.435512,1605269.1,808.346754,103466.2,6349.195157,519901.00,1639.259095,109818.9,9033.696968,15623.40,10656.460566,54218.0,5627.455033,275827.7,1935.797366,506570.1,1119.811710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,2021-10-26,1608052.10,558.708368,1865839.8,385.508099,1755043.6,894.931198,109742.8,6401.588378,659199.80,1392.728329,61851.1,14269.704581,117700.11,4044.794555,52296.0,5360.343372,121020.5,3971.806504,196670.5,2818.614261
79,2021-10-27,1570622.70,531.337939,1864217.3,386.123104,1628062.3,934.278134,126170.9,6398.261953,642509.80,1321.672536,64848.3,13419.232424,119736.46,3892.642032,55649.0,5201.431957,142164.1,3712.621442,187519.6,3035.949895
80,2021-10-28,1612247.30,556.408045,1674750.8,419.440625,1615172.4,879.953256,119015.0,5774.265681,607406.85,1300.716197,64702.3,12048.405868,123767.59,3964.834073,48883.2,5617.076398,147882.0,3335.363574,147721.0,3608.149011
81,2021-10-29,1592272.90,538.959291,1940111.6,429.458454,1487353.2,896.397056,86053.7,5609.026352,577974.30,1315.346080,70524.5,10837.457961,127396.90,3824.269515,53240.8,5693.069807,151865.4,3524.282858,251039.4,3475.340006


In [ ]:
tr.to_csv('/content/drive/MyDrive/Capstone/Data/test2.csv', index=False)

In [ ]:
qwert = pd.read_csv('/content/drive/MyDrive/Capstone/Data/ten_pum_train_onebon.csv')

In [ ]:
qwert

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg)
0,2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,15019.0,5475.0,92334.0,1704.0,4374.9,13242.0,16550.5,2339.0,10528.0,1729.0,3853.0,3703.0,30950.0,1621.0
2,2016-01-03,일요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-04,월요일,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,141638.0,5210.0,994328.1,1716.0,122613.5,9923.0,427435.1,2153.0,82113.5,3960.0,104930.3,4871.0,291057.0,1834.0
4,2016-01-05,화요일,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,126207.8,5387.0,787716.0,1715.0,79055.9,9529.0,334636.8,2220.0,80144.0,3333.0,100699.5,5129.0,194626.5,1833.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2034,2021-07-27,화요일,865125.6,601.0,964146.2,485.0,1562374.7,607.0,111638.2,5816.0,535457.2,725.0,90178.7,4071.0,21938.6,7427.0,50378.1,1947.0,185578.5,1722.0,260965.8,1705.0
2035,2021-07-28,수요일,832213.3,585.0,873415.5,579.0,1524724.7,646.0,103044.8,6342.0,474284.6,706.0,86493.4,4148.0,19731.9,7556.0,46993.5,1675.0,196840.8,1734.0,349643.6,1757.0
2036,2021-07-29,목요일,782651.1,596.0,1138537.6,469.0,1773969.0,614.0,89687.6,6613.0,488847.6,794.0,81400.8,4564.0,20553.9,7429.0,44376.0,1929.0,185165.8,1696.0,276189.7,1796.0
2037,2021-07-30,금요일,742723.2,659.0,980013.7,464.0,1749363.9,636.0,63553.2,5817.0,479356.5,777.0,85792.9,5417.0,19469.8,8476.0,43582.0,2183.0,198537.9,1701.0,385327.1,1722.0


In [ ]:
train2[212:]

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg)
212,2021-08-01,일요일,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
213,2021-08-02,월요일,998824.04,763.819668,1140263.3,610.960763,1513622.4,665.459000,114825.2,6251.908823,506513.8,885.820829,115570.4,5330.649812,19810.86,8397.272405,49062.1,3063.390540,229907.7,1767.123850,578291.0,1340.590481
214,2021-08-03,화요일,820255.49,905.244796,1118840.6,567.260575,1589983.6,677.135062,136224.1,6121.899451,463368.7,988.611523,84821.9,4918.613153,18438.90,7488.402182,47859.5,2795.417002,143655.8,2109.013468,408103.6,1245.547699
215,2021-08-04,수요일,1000020.00,779.683542,1178932.3,492.572093,1589479.2,771.450325,97192.0,6716.579883,445159.0,1259.425235,88627.9,5448.249468,17413.44,7900.040486,44720.0,3035.610555,163340.3,2250.771383,448618.3,1231.644451
216,2021-08-05,목요일,930687.10,821.295017,1159449.5,500.682069,1981966.3,776.632317,85013.5,6125.134337,523013.3,1307.881673,92323.2,6897.028797,15223.80,8821.209488,49449.7,3597.859744,181197.5,2543.051736,410853.3,1195.879868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,2021-11-26,금요일,3322119.50,855.526302,2848866.7,435.322178,1420063.1,824.988797,77781.7,6688.693844,606203.2,1159.468442,67183.0,5270.268357,229238.05,2226.226340,82529.9,6738.824341,165520.6,2108.242279,231763.5,2675.807783
330,2021-11-27,토요일,3099051.25,844.090363,2657614.8,378.614620,1354649.4,772.076177,77304.3,6706.403292,713593.0,1086.698594,67172.0,5169.389046,221894.62,2275.984041,85130.5,8013.229947,122186.3,2080.165043,196120.0,2294.284622
331,2021-11-28,일요일,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
332,2021-11-29,월요일,2856016.45,801.194158,2209466.0,380.848191,1910143.7,816.013853,84870.7,6332.339559,682166.3,1150.958392,79927.8,5897.250343,275299.90,2294.806424,82265.7,7140.467011,203478.4,2274.810506,232403.5,2486.894345


In [ ]:
import datetime #20/9/29~20/11/5
days = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
df=[]

for i in range (1, 32):
  nalja2 = days[datetime.date(2021, 1, i).weekday()]
  df.append(nalja2)

for i in range (1, 29):
  nalja2 = days[datetime.date(2021, 2, i).weekday()]
  df.append(nalja2)

for i in range (1, 32):
  nalja2 = days[datetime.date(2021, 3, i).weekday()]
  df.append(nalja2)

for i in range (1, 31):
  nalja2 = days[datetime.date(2021, 4, i).weekday()]
  df.append(nalja2)

for i in range (1, 32):
  nalja2 = days[datetime.date(2021, 5, i).weekday()]
  df.append(nalja2)

for i in range (1, 31):
  nalja2 = days[datetime.date(2021, 6, i).weekday()]
  df.append(nalja2)

for i in range (1, 32):
  nalja2 = days[datetime.date(2021, 7, i).weekday()]
  df.append(nalja2)

for i in range (1, 32):
  nalja2 = days[datetime.date(2021, 8, i).weekday()]
  df.append(nalja2)

for i in range (1, 31):
  nalja2 = days[datetime.date(2021, 9, i).weekday()]
  df.append(nalja2)

for i in range (1, 32):
  nalja2 = days[datetime.date(2021, 10, i).weekday()]
  df.append(nalja2)

for i in range (1, 31):
  nalja2 = days[datetime.date(2021, 11, i).weekday()]
  df.append(nalja2)

df

['금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',
 '목요일',
 '금요일',
 '토요일',
 '일요일',
 '월요일',
 '화요일',
 '수요일',


In [ ]:
df = pd.DataFrame(df)
df.shape

(334, 1)

In [ ]:
train2.columns

Index(['date', '배추_거래량(kg)', '배추_가격(원/kg)', '무_거래량(kg)', '무_가격(원/kg)',
       '양파_거래량(kg)', '양파_가격(원/kg)', '마늘_거래량(kg)', '마늘_가격(원/kg)', '대파_거래량(kg)',
       '대파_가격(원/kg)', '깻잎_거래량(kg)', '깻잎_가격(원/kg)', '시금치_거래량(kg)',
       '시금치_가격(원/kg)', '미나리_거래량(kg)', '미나리_가격(원/kg)', '파프리카_거래량(kg)',
       '파프리카_가격(원/kg)', '토마토_거래량(kg)', '토마토_가격(원/kg)'],
      dtype='object')

In [ ]:
train2['요일'] = df

In [ ]:
train2 = train2[['date', '요일', '배추_거래량(kg)', '배추_가격(원/kg)', '무_거래량(kg)', '무_가격(원/kg)',
       '양파_거래량(kg)', '양파_가격(원/kg)', '마늘_거래량(kg)', '마늘_가격(원/kg)', '대파_거래량(kg)',
       '대파_가격(원/kg)', '깻잎_거래량(kg)', '깻잎_가격(원/kg)', '시금치_거래량(kg)',
       '시금치_가격(원/kg)', '미나리_거래량(kg)', '미나리_가격(원/kg)', '파프리카_거래량(kg)',
       '파프리카_가격(원/kg)', '토마토_거래량(kg)', '토마토_가격(원/kg)']]

In [ ]:
hap2 = train2[212:]

In [ ]:
hap2.shape

(122, 22)

In [ ]:
qwert.shape

(2039, 22)

In [ ]:
data = pd.concat([qwert, hap2]) #인덱스 번호까지 합쳐지게

In [ ]:
data.to_csv('/content/drive/MyDrive/Capstone/Data/Nong_Dataset.csv', index=False)

In [ ]:
train2.to_csv('/content/drive/MyDrive/Capstone/Data/test.csv', index=False)

# <font color = 'lime'> **test_files 코드** <font>

In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Capstone/Data/test_yesterday.csv')

In [ ]:
data.shape #8월 1일 ~ 10월 31일(가격이 0인날 제외 - 일요일, 연휴)

(336, 22)

In [ ]:
data['date'][335] #데이터 잘 들어왔는지 마지막 행 확인

'2021-12-02'

In [ ]:
for i in range (334, 336):  #42

   test = data[0:i+1]

   nalja = data['date'][i] #날짜 불러오기 2021-08-24
   
   print(test) #csv 저장하기 전에 확인
   
   test.to_csv('/content/drive/MyDrive/Capstone/Data/test_files/' + f'test_{nalja}' + '.csv', index=False)

           date   요일  배추_거래량(kg)  ...  파프리카_가격(원/kg)  토마토_거래량(kg)  토마토_가격(원/kg)
0    2021-01-01  금요일        0.00  ...       0.000000          0.0      0.000000
1    2021-01-02  토요일    24401.50  ...    4884.721875      25290.0   1914.895216
2    2021-01-03  일요일        0.00  ...       0.000000          0.0      0.000000
3    2021-01-04  월요일   859651.80  ...    4515.337752     393979.0   2098.278649
4    2021-01-05  화요일   889332.20  ...    5515.671581     228576.5   2092.442574
..          ...  ...         ...  ...            ...          ...           ...
330  2021-11-27  토요일  3099051.25  ...    2080.165043     196120.0   2294.284622
331  2021-11-28  일요일        0.00  ...       0.000000          0.0      0.000000
332  2021-11-29  월요일  2856016.45  ...    2274.810506     232403.5   2486.894345
333  2021-11-30  화요일  2185300.05  ...    2225.523196     176793.4   2358.261281
334  2021-12-01  수요일  1517157.50  ...    2134.811434     190878.7   2507.076484

[335 rows x 22 columns]
           date

# <font color = 'lime'> **private용 test_files 코드** <font>

In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Capstone/private_data/private_data.csv')

In [ ]:
data.shape #2020/11/6 ~ 2021/8/23

(291, 44)

In [ ]:
data['date'][290] #데이터 잘 들어왔는지 마지막 행 확인

'2021-08-23'

In [ ]:
for i in range (0, 291):  #42

   test = data[0:i+1]

   nalja = data['date'][i] #날짜 불러오기 2021-08-24

   #print(test) #csv 저장하기 전에 확인
   
   test.to_csv('/content/drive/MyDrive/Capstone/private_data/test_files/' + f'test_{nalja}' + '.csv', index=False)